In [1]:
import sys
import os
import dtale
import dtale.app as dtale_app
dtale_app.JUPYTER_SERVER_PROXY = True
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config
config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}
config = Config(config=config_info)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

A configuration file has been created for you in 
/home/canalyst-candas/canalyst/keys.json. For Excel file downloads and scenario 
mapping, you'll need to add an API key to the 'canalyst_api_key' 
value. Visit https://app.canalyst.com/settings/api-tokens to create 
and retrieve your API key. You will also need to retrieve your 
S3 Access ID and Secret Key and fill in 'canalyst_s3_id' and 
'canalyst_s3_key' with those values, respectively. If using a 
Jupyter Notebook, stop and restart the notebook for the changes to 
take effect. If using a Python/iPython session, quit the current 
session and start a new one.


## JPM Read Through for credit and debit card volumes to V and MA

### First we search for common time series among JPM V and MA with 'credit card sales volume debit payments US'

In [2]:
canalyst_search = cd.Search(config)
df = canalyst_search.search_time_series(ticker=['JPM US','V US','MA US'],
                                        unit_type='currency',
                     time_series_description='credit card sales volume debit payments volume US')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>3]

Building search index...
No.of.unique tickers:  3
No.of.unique time series:  399


In [3]:
df

,ticker,time_series_description,time_series_name,search_matches
50289,JPM US,"Debit and credit card sales volumes, bn",z_3FRM7X0148_MO_OS_Debitandcreditcardsalesvolumes,5
3386,V US,"US consumer debit payments volume, bn",z_2CRCA60110_MO_OS_usconsumerdebitpaymentsvolume,4
3385,V US,"US consumer credit payments volume, bn",z_2CRCA60110_MO_OS_usconsumercreditpaymentsvolume,4
6372,MA US,"Total MasterCard debit program cards, mm cards",z_GSYQ3D0168_MO_OS_TotalMasterCarddebitprogramcards_1,4
6369,MA US,"Total MasterCard credit and charge programs cards, mm cards",z_GSYQ3D0168_MO_OS_TotalMasterCardcreditandchargeprogramscards_3,4
6378,MA US,"Commercial credit and debit cards, mm cards",z_GSYQ3D0168_MO_OS_Commercialcreditanddebitcards,4
50288,JPM US,"Debit card sales volume, bn",z_3FRM7X0148_MO_OS_Debitcardsalesvolume,4
50287,JPM US,"Credit card sales volume, bn",z_3FRM7X0148_MO_OS_Creditcardsalesvolume,4


### Now that we have our series, let's pull the data from Canalyst

In [4]:
ticker = "JPM US"

In [5]:
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)

In [6]:
df_jpm_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_Creditcardsalesvolume')[['ticker','time_series_name','period_end_date','value']]

df_jpm_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_Debitcardsalesvolume')[['ticker','time_series_name','period_end_date','value']]
 


### Next we get Visa data from Canalyst

In [7]:
ticker = "V US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_visa_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_2CRCA60110_MO_OS_usconsumercreditpaymentsvolume')[['ticker','time_series_name','period_end_date','value']]

df_visa_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_2CRCA60110_MO_OS_usconsumerdebitpaymentsvolume')[['ticker','time_series_name','period_end_date','value']]


### We need to do a deeper search on Mastercard time series, add 'programs' and 'unitedstates' to our search

In [8]:
#add 'programs' and 'unitedstates' to our search
df = canalyst_search.search_time_series(ticker=['MA US'],
                                        unit_type='currency',
                     time_series_description='charge debit programs volume UnitedStates')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>2]
df

No.of.unique tickers:  1
No.of.unique time series:  134


,ticker,time_series_description,time_series_name,search_matches
6261,MA US,"Worldwide less United States debit programs purchase volume, bn",z_GSYQ3D0168_MO_OS_WorldwidelessUnitedStatesdebitprogramspurchasevolume_3,3
6317,MA US,"Worldwide less United States debit programs cash volume, bn",z_GSYQ3D0168_MO_OS_WorldwidelessUnitedStatesdebitprogramscashvolume_3,3
6258,MA US,"Worldwide less United States credit and charge programs purchase volume, bn",z_GSYQ3D0168_MO_OS_WorldwidelessUnitedStatescreditandchargeprogramspurchasevolume_3,3
6314,MA US,"Worldwide less United States credit and charge programs cash volume, bn",z_GSYQ3D0168_MO_OS_WorldwidelessUnitedStatescreditandchargeprogramscashvolume_3,3
6262,MA US,"United States debit programs purchase volume, bn",z_GSYQ3D0168_MO_OS_UnitedStatesdebitprogramspurchasevolume_3,3
6318,MA US,"United States debit programs cash volume, bn",z_GSYQ3D0168_MO_OS_UnitedStatesdebitprogramscashvolume_3,3
6259,MA US,"United States credit and charge programs purchase volume, bn",z_GSYQ3D0168_MO_OS_UnitedStatescreditandchargeprogramspurchasevolume_3,3
6315,MA US,"United States credit and charge programs cash volume, bn",z_GSYQ3D0168_MO_OS_UnitedStatescreditandchargeprogramscashvolume_3,3
6260,MA US,"Total MasterCard credit and charge programs purchase volume, bn",z_GSYQ3D0168_MO_OS_TotalMasterCardcreditandchargeprogramspurchasevolume_1,3
6316,MA US,"Total MasterCard credit and charge programs cash volume, bn",z_GSYQ3D0168_MO_OS_TotalMasterCardcreditandchargeprogramscashvolume_2,3


### Given the better time series matches, same data pull for MA

In [9]:
ticker = "MA US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_mastercard_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_GSYQ3D0168_MO_OS_UnitedStatescreditandchargeprogramspurchasevolume_3')[['ticker','time_series_name','period_end_date','value']]
df_mastercard_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_GSYQ3D0168_MO_OS_UnitedStatesdebitprogramspurchasevolume_3')[['ticker','time_series_name','period_end_date','value']]


### Now we use the regress_dataframe function from Candas DataReader
#### R squared for JPM vs V and MA, first by credit then by debit in turn

In [10]:
df = pd.merge(df_jpm_credit_volume,df_visa_credit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

#Visa fit is not as good because:
#for Visa, "Service revenues in a given quarter are assessed based on nominal payments volume in the prior quarter."
#footnote 1 p 37 Visa annual report

Dep. Variable:,value_y,R-squared:,0.650
Model:,OLS,Adj. R-squared:,0.634
Method:,Least Squares,F-statistic:,40.79
Date:,"Wed, 12 Jan 2022",Prob (F-statistic):,2.00e-06
Time:,18:25:10,Log-Likelihood:,-613.69
No. Observations:,24,AIC:,1231.
Df Residuals:,22,BIC:,1234.
Df Model:,1,,
Covariance Type:,nonrobust,,


In [11]:
df = pd.merge(df_jpm_credit_volume,df_mastercard_credit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.968
Model:,OLS,Adj. R-squared:,0.967
Method:,Least Squares,F-statistic:,671.1
Date:,"Wed, 12 Jan 2022",Prob (F-statistic):,5.62e-18
Time:,18:25:10,Log-Likelihood:,-572.16
No. Observations:,24,AIC:,1148.
Df Residuals:,22,BIC:,1151.
Df Model:,1,,
Covariance Type:,nonrobust,,


In [12]:
df = pd.merge(df_jpm_debit_volume,df_visa_debit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.951
Model:,OLS,Adj. R-squared:,0.949
Method:,Least Squares,F-statistic:,408.7
Date:,"Wed, 12 Jan 2022",Prob (F-statistic):,3.03e-15
Time:,18:25:10,Log-Likelihood:,-578.78
No. Observations:,23,AIC:,1162.
Df Residuals:,21,BIC:,1164.
Df Model:,1,,
Covariance Type:,nonrobust,,


In [13]:
df = pd.merge(df_jpm_debit_volume,df_mastercard_debit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.967
Model:,OLS,Adj. R-squared:,0.965
Method:,Least Squares,F-statistic:,613.7
Date:,"Wed, 12 Jan 2022",Prob (F-statistic):,4.99e-17
Time:,18:25:10,Log-Likelihood:,-554.37
No. Observations:,23,AIC:,1113.
Df Residuals:,21,BIC:,1115.
Df Model:,1,,
Covariance Type:,nonrobust,,
